In [1]:
import os
import tkinter as tk
from PIL import Image,ImageDraw
from joblib import dump, load
from sklearn.svm import SVC
import numpy as np
import torch

In [2]:
class DigitPainter(tk.Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        
        self.lastX, self.lastY = 0, 0        
        self.pencil=45        
        self.imgside=448
        
        self.h=int((self.pencil-1)/2)
        self.create_widgets()
        self.svmclass= None
        self.rdfclass= None
        self.annclass= None
        self.knnclass= None
        self.loadPredictors()
        
    def create_widgets(self):
        self.master.geometry('448x512')  # 448=28x16
        self.master.title("Drawing window")

        self.img = Image.new(mode='L',size=(self.imgside,self.imgside),color=(0)) # 'L' means 8-bits        
        self.painter=ImageDraw.Draw(self.img)
        self.canvas= tk.Canvas(self.master,width=self.imgside,height=self.imgside,bg="black")
               
        self.canvas.bind( "<Button-1>", self.get_x_and_y )
        self.canvas.bind( "<B1-Motion>", self.draw_box )
        self.canvas.pack() #expand = YES, fill = BOTH)
        
        # Botones y acciones
        self.frame = tk.Frame(self.master)
        self.frame.pack()

        self.resetButton=tk.Button(self.frame,text="Reset",command= self.erase_canvas) 
        self.resetButton.pack(side= tk.LEFT)
        
        self.svmButton=tk.Button(self.frame,text="SVM",command= self.predictSVM)
        self.svmButton.pack(side= tk.RIGHT)

        self.annButton=tk.Button(self.frame,text="ANN",command= self.predictANN)
        self.annButton.pack(side= tk.RIGHT)

        self.knnButton=tk.Button(self.frame,text="KNN",command= self.predictKNN)
        self.knnButton.pack(side= tk.RIGHT)

        self.rdfButton=tk.Button(self.frame,text="RDF",command= self.predictRDF)
        self.rdfButton.pack(side= tk.RIGHT)
        
        
    # Called when the button is pressed the first time
    def get_x_and_y(self,event):
        self.lastX,self.lastY = event.x, event.y

        self.svmButton['state']=tk.NORMAL
        self.annButton['state']=tk.NORMAL
        self.knnButton['state']=tk.NORMAL
        self.rdfButton['state']=tk.NORMAL

        self.resetButton['state']=tk.NORMAL
        
    # Called on a drag event
    def draw_box(self,event):
        
        self.canvas.create_line((self.lastX,self.lastY,event.x,event.y),fill='white',width=self.pencil)
        self.canvas.create_oval((event.x-self.h,event.y-self.h,event.x+self.h,event.y+self.h),
                                fill='white',outline='white')
        
        self.painter.line([(self.lastX,self.lastY),(event.x,event.y)],fill='white',width=self.pencil)
        self.painter.ellipse([(event.x-self.h,event.y-self.h),(event.x+self.h,event.y+self.h)],
                               fill='white',outline='white')        
        
        self.lastX,self.lastY=event.x,event.y

        
    #Encargado de borrar todo (para bóton)
    def erase_canvas(self):
    
        self.canvas.delete('all')  
        
        self.svmButton['state']=tk.DISABLED
        self.annButton['state']=tk.DISABLED
        self.knnButton['state']=tk.DISABLED
        self.rdfButton['state']=tk.DISABLED
        
        self.resetButton['state']=tk.DISABLED
    
        self.painter.rectangle([0,0,self.imgside,self.imgside],fill='black',outline='black')
        
    def getMNISTImage(self):
        return np.array(self.img.resize((28,28),Image.ANTIALIAS))
        
    # ---------------------------
    # Your code below this point
        
    def loadPredictors(self):
        
        print("Loading predictors...")
        
        # ADD YOUR CODE HERE TO LOAD ALL FOUR PREDICTORS
        self.svmclass=load("svmclass.joblib")
        self.rdfclass=load("rdfclass.joblib")
        self.knnclass=load("knnclass.joblib")
        self.annclass = torch.jit.load('model_scripted.pt')
        self.annclass.eval()
   
        
    def predictSVM(self):
        print("Predict SVM called")
        img=self.getMNISTImage()
        
        # ADD YOUR CODE HERE TO PREDICT WITH SVM
        h,w=img.shape
        sample = img.reshape((1,w*h)).astype(float)/255 # Esto es una matrix de m x 784
        
        predict = self.svmclass.predict(sample)
        
        print(predict)
        
    def predictANN(self):
        print("Predict ANN called")
        img=self.getMNISTImage()

        # ADD YOUR CODE HERE TO PREDICT WITH ANN
        h,w=img.shape
        sample = img.reshape((1,w*h)).astype(float)/255 # Esto es una matrix de m x 784
        sample = torch.FloatTensor(sample)

        self.annclass.eval()

        predict = self.annclass.forward(sample)
        buffer2 = torch.max(predict, 1) [1]
        buffer2=buffer2.detach().numpy()

        print(buffer2)
        
        
    def predictKNN(self):
        print("Predict KNN called")
        img=self.getMNISTImage()
        
        # ADD YOUR CODE HERE TO PREDICT WITH KNN
        h,w=img.shape
        sample = img.reshape((1,w*h)).astype(float)/255 # Esto es una matrix de m x 784
    
        predict = self.knnclass.predict(sample)
        
        print(predict)        
    def predictRDF(self):
        print("Predict RDF called")
        img=self.getMNISTImage()
        
        # ADD YOUR CODE HERE TO PREDICT WITH RDF        
        h,w=img.shape
        sample = img.reshape((1,w*h)).astype(float)/255 # Esto es una matrix de m x 784
        
        predict = self.rdfclass.predict(sample)
        
        print(predict)
        
        
        

In [6]:
root = tk.Tk()
painter=DigitPainter(master=root)
painter.mainloop()

Loading predictors...
